# "Why Should I Trust You?" Explaining the Predictions of Any Classifier

2018.07.15. <br>
Jaehwi Park

## 1. Introduction

> _ML영역에서 인간이 하는 일도 중요한데.. 간과당하고 있다고 주장합니다._
> - 모형을 직접 사용하고
> - 모형을 프로덕트에 배치합니다.

<br>
> _모형에 대한 신뢰(trust)는 두 가지로 구별돼야 합다고 합니다._
> - 모형의 개별 Prediction 결과
> - 모형 자체

<br>
> _본 논문에서는 다음을 제공합니다._
> - 개별 Prediction에 대한 설명
> - 모형 전체를 대표할 수 있는 유의미한 Instance를 선택하는 방법과 그에 대한 설명

<br>
> _Main Contribution !!_
> - LIME:  _Any Classifier or Regressor_의 예측에 대한 설명 알고리즘입니다. (by approximating it locally with an interpretable model)
> - SP-LIME: 대표 Instances를 선택하는 방법입니다.

## 2. The Case for Explanations

Input과 Output(Prediction) 사이의 관계를 보여줄 때 질적 평가가 가능합니다. 그리고 모형에 대한 결과를 도메인 지식을 미리 가지고 있는 전문가가 납득할 수 있을 때, 모형에 대한 신뢰가 가능합니다.

![Figure1](./Figure1.png)

> _모형이 잘못될 수 있는 여지들입니다._
> - Data Leakage: 중요한데 없거나 전혀 중요치 않은데 모형에서 heavily rely on
> - Dataset Shift: training <-> test 간의 변화

<br>
> _Desired Characteristics for Explainers_
> - Interpretable
> - Local Fidelity
> - Model-agnostic
> - global perspective



## 3. Local Interpretable Model-Agnostic Explanations (LIME)

### 3.1 Interpretable Data Representations

> 정말 Raw input과 인간이 이해할 수 있는 representations는 차이가 있습니다. <br>
> 따라서 다음 두 개는 구분이 필요합니다.
> - features
> - Interpretable Data Representations: 사람이 이해할 수 있는 features의 조합

<br>
> Notation
> - Original representation of an instance: $x \in \mathbb{R}^d$
> - interpretable representation: $x' \in \{0,1\}^{d'}$  -> 특정 단어의 조합 또는 특정 patch의 유무(Binary)

### 3.2 Fidelity - Interpretability Trade-off

> - 설명을 모형 $g \in G$로 합니다. G는 전통적인 해석가능한 모형을 의미합니다. (Linear Model, Decision Tree, or Falling Rule Lists)
> - $g \in G$의 도메인은 $\{0,1\}^{d'}$ 입니다. -> g가 "interpretable components"의 유무를 바탕으로 작동합니다.
> - 모든 $g \in G$가 해석가능하지 않으므로 $\Omega (g)$로 g의 복잡도를 정의합니다. 복잡도가 증가하면 해석가능도가 떨어집니다.
>> - Decision Tree라면 Node의 깊이가 복잡도에 해당하고
>> - Linear Model의 경우는 Non-zero weights의 수가 복잡도에 해당할 것입니다.
> - 설명할 모형 $f: \mathbb{R}^d \to \mathbb{R}$
> - 거리측도 from instance z to input x: $\pi _x(z)$
> - 설명모형 g가 unfaithful한 정도: $\mathcal{L} (f,g,\pi _x)$ -> 그러므로 minimize 해야함
> - $\xi (x) = argmin_{g \in G} \mathcal{L} (f,g,\pi _x) + \Omega (g)$

### 3.3 Sampling for Local Exploration

> - f에 상관없이 $\mathcal{L} (f,g,\pi _x)$를 최소화 하고 싶음
> - $\pi _x$에 가중치를 둔 샘플링으로 $\mathcal{L} (f,g,\pi _x)$를 근사합니다.
> - nonzero elements of x'를 uniform random하게 뽑아서 z'를 만듭니다. (where the number of such draws is also uniformly sampled)
> - $z' \in \{0,1\} ^{d'}$를 다시 원래의 representation $z \in R^d$로 돌려서 f(z)를 구합니다.
> - 이러한 z'들 및 f(z)들의 결과(label) set을 가지고 위의 $\xi (x)$를 최적화합니다.

![Figure3](./Figure3.png)

### 3.4 Sparse Linear Explanations

![Algorithm1](Algorithm1.png)

> - G를 Linear Model로 정의하면, $g(z') = w_g \cdot z'$
> - $\pi _x(z) = exp(-D(x,z)^2/\sigma ^2)$
>> - $\pi _x(z)$는 Exponential Kernel 입니다.
>> - D는 Distance Function (cosine distance for text, L2 distance for Image)
>> - $\sigma$는 width입니다.
> - $\mathcal{L} (f,g,\pi _x) = \sum_{\substack{z,z' \in \mathcal{Z}}} \pi_x(z) (f(z) - g(z'))^2$
> - 복잡도 정의
>> - Text 분류에서는 단어들의 수 상한을 K로 정의함
>> - Image에서는 "super-pixels"를 바탕으로 K개의 feature들을 LASSO 정규화를 통해 정의함
![superpixel](./superpixel.png)
> - 개별 Instance에 대한 해석이므로 Dataset 크기와는 무관하고, f(x) computing 시간 및 샘플 크기 N에 비례하여 시간이 소요된다.

<br>
> 단점 & Future Works
> - interpretable representation이 충분히 강력하지 (적절치) 못할 수 있음. ex) super pixel로는 색의 영향 파악이 불가
> - 모형이 매우매우매우 복잡하다면 선형설명기(G)로는 충분치 않을 수 있음

## 4. Submodular Pick for Explaining Models

> - B instances 를 선별하여 모형 전체를 대변해보자 (Budget)
> - global behavior를 이해하기 위해서는 _pick step_이 다양하고 모형을 대표하는 instance를 선별해야 한다.
> - n개의 instance가 뽑히면, 최대 n x d'의 설명이 가능해진다.
>> - instance $x_i$와 그에 대한 설명 $g_i = \xi(x_i)$이 있을 때
>> - 행렬 $\mathcal{W}_{ij}= |w_{g_{ij}}|$ 생성이 가능하다.
>> - 행렬 $\mathcal{W}_{ij}$ 에서 열마다 Global Importance $I_j$를 계산할 수 있다.
![Figure5](Figure5.png)
>> - Text의 경우는 위 그림처럼 $I_j = \sqrt{\displaystyle\sum_{i=1}^{n} \mathcal{W}_{ij}}$
>> - Image의 경우는 super-pixel간 비교할 수 있는 성질을 찾아야 하는데 ... remain future work ^^
> - 아래의 알고리즘은 계산 복잡도가 NP-hard 이므로 Greedy Algorithm을 적용합니다.
> ![Algorithm2](Algorithm2.png)